# Crypto_Scraper

## Import 

In [5]:
import praw
import pandas as pd
from datetime import datetime
import os
import psaw as ps
from dotenv import load_dotenv

In [6]:
# Lade die .env-Datei
dotenv_loaded = load_dotenv("zugang_reddit.env")  # Falls die Datei anders heißt, anpassen
# Prüfe, ob die Datei geladen wurde
print(f".env geladen? {dotenv_loaded}")


.env geladen? True


In [7]:
reddit = praw.Reddit(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    user_agent=os.getenv("USER_AGENT")
)

print("Reddit API erfolgreich verbunden!")


Reddit API erfolgreich verbunden!


In [8]:
try:
    subreddit = reddit.subreddit("CryptoCurrency")
    for post in subreddit.hot(limit=5):
        print(f"Title: {post.title}, Score: {post.score}")
except Exception as e:
    print(f"Fehler beim Abrufen der Subreddit-Daten: {e}")


Title: Kraken Pay AMA with Jared Blake, $5000 USDG* in prizes up for grabs!, Score: 21
Title: Daily Crypto Discussion - January 29, 2025 (GMT+0), Score: 34
Title: Crypto will change you, Score: 727
Title: They say humor can be a coping mechanism for trauma., Score: 133
Title: Bill Zanker, the man behind Trump's $8B crypto coin, tried to throw thousands of dollar bills from Empire State Building, Score: 217


## Liste der Cryptos

In [9]:
cryptos = [
    ("Cardano", "ADA"),
    ("Solana", "SOL"),
    ("Binance Coin", "BNB"),
    ("XRP", "XRP"),
    ("Polkadot", "DOT"),
    ("Dogecoin", "DOGE")
]


In [10]:
subreddits = [
    "CryptoCurrency",  # Allgemeine Diskussionen über Kryptowährungen
    "CryptoMarkets",   # Diskussionen über den Kryptomarkt und Preisbewegungen
    "CryptoTrading",   # Fokus auf Trading-Strategien und Analysen
    "Altcoin",         # Diskussionen über Altcoins (alle Kryptowährungen außer Bitcoin)
    "DeFi",            # Decentralized Finance (DeFi) und Projekte
    "BitcoinBeginners",# Für Anfänger in der Krypto-Welt
    "cryptotechnology", # Fokus auf die zugrunde liegende Blockchain-Technologie
    "cryptocurrencies", # Allgemeine Diskussionen über Kryptowährungen
    "Satoshistreetsbets", # Krypto-Wetten und Spekulationen
    "Binance"        # Diskussionen über die Binance-Plattform  
]

## Scrape

In [11]:
# Funktion, um Posts und Kommentare zu scrapen
def scrape_reddit(start_timestamp, end_timestamp, mode="initial"):
    posts = []
    comments = []
    post_ids = set()  # Set zum Speichern der Post-IDs

    for crypto_name, crypto_symbol in cryptos:
        for subreddit_name in subreddits:
            subreddit = reddit.subreddit(subreddit_name)
            print(f"Suche Posts zu {crypto_name} ({crypto_symbol}) in r/{subreddit_name}...")

            # Suche nach Namen und Kürzel
            for search_term in [crypto_name, crypto_symbol]:
                for post in subreddit.search(query=search_term, sort="new", limit=None):
                    if start_timestamp <= post.created_utc <= end_timestamp and post.id not in post_ids:
                        post_ids.add(post.id)
                        posts.append({
                            'crypto': crypto_name,
                            'search_term': search_term,
                            'subreddit': subreddit_name,
                            'post_id': post.id,
                            'title': post.title,
                            'author': str(post.author),
                            'created_utc': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                            'score': post.score,
                            'num_comments': post.num_comments,
                            'selftext': post.selftext
                        })
                        print(f"Gefunden in r/{subreddit_name}: {post.title} (Suchbegriff: {search_term})")

                        # Kommentare sammeln
                        post.comments.replace_more(limit=0)
                        for comment in post.comments.list():
                            comments.append({
                                'post_id': post.id,
                                'comment_id': comment.id,
                                'author': str(comment.author),
                                'created_utc': datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                                'score': comment.score,
                                'body': comment.body
                            })

    # In DataFrames umwandeln
    df_posts = pd.DataFrame(posts)
    df_comments = pd.DataFrame(comments)

    # DataFrames zurückgeben
    print(f"Scrape abgeschlossen: {len(df_posts)} Posts und {len(df_comments)} Kommentare gefunden.")
    return df_posts, df_comments

# **Wöchentlicher Scrape**
now = int(datetime.now().timestamp())  # Aktueller Zeitstempel
last_week = now - 7 * 24 * 60 * 60  # 7 Tage zurück
print("Starte den wöchentlichen Scrape...")
df_posts_weekly, df_comments_weekly = scrape_reddit(last_week, now, mode="weekly")

# Beispiel: Lokale Weiterverarbeitung
print("Daten können jetzt in der Pipeline bereinigt werden...")

Starte den wöchentlichen Scrape...
Suche Posts zu Cardano (ADA) in r/CryptoCurrency...
Gefunden in r/CryptoCurrency: Cardano Set to Achieve Full Decentralized Governance with Wednesday’s Plomin Hard Fork (Suchbegriff: Cardano)
Gefunden in r/CryptoCurrency: Understanding the "inverse r/cryptocurrency" trading strategy. Is this sub consistently wrong? How it works, how to exploit it, and the underlying reason driving it: (Suchbegriff: Cardano)
Gefunden in r/CryptoCurrency: Tuttle Capital Seeks SEC Approval for First-Ever Chainlink, Cardano, and Polkadot ETFs (Suchbegriff: Cardano)
Gefunden in r/CryptoCurrency: Why are there no solid projects built on top layer 1s like sui and solana? (Suchbegriff: Cardano)
Gefunden in r/CryptoCurrency: Replacing ETC (Suchbegriff: Cardano)
Gefunden in r/CryptoCurrency: Does market cap determine ATH, or does price? (Suchbegriff: ADA)
Suche Posts zu Cardano (ADA) in r/CryptoMarkets...
Gefunden in r/CryptoMarkets: Portfolio Advice (Suchbegriff: Cardano)
Gefu

## Datenbereinigung 

In [12]:
def clean_data(df_posts, df_comments):
    # 1. Duplikate entfernen
    df_posts = df_posts.drop_duplicates(subset=["post_id"])
    df_comments = df_comments.drop_duplicates(subset=["comment_id"])
    
    # 2. Fehlende Werte behandeln
    df_posts['selftext'] = df_posts['selftext'].fillna('')  # Fehlende Posttexte auffüllen
    df_comments['body'] = df_comments['body'].fillna('')  # Fehlende Kommentare auffüllen
    
    # 3. Zeitstempel konvertieren
    df_posts['created_utc'] = pd.to_datetime(df_posts['created_utc'])
    df_comments['created_utc'] = pd.to_datetime(df_comments['created_utc'])
    
    # 4. Filterung nach Qualität (z. B. Spam entfernen)
    df_posts = df_posts[df_posts['score'] > 0]  # Posts mit negativem Score entfernen
    df_comments = df_comments[df_comments['score'] > 0]  # Kommentare mit negativem Score entfernen
    
    return df_posts, df_comments


Funktion zur bereinigung auf die Daten anwenden

In [13]:
# Bereinigen der Daten
df_posts_clean, df_comments_clean = clean_data(df_posts_weekly, df_comments_weekly)

# Überprüfen, wie viele Einträge übrig sind
print(f"Bereinigte Posts: {len(df_posts_clean)}")
print(f"Bereinigte Kommentare: {len(df_comments_clean)}")

Bereinigte Posts: 99
Bereinigte Kommentare: 5497


## Google Drive Export

Pfad

In [14]:
# Definiere den Speicherpfad zu deinem lokalen Google Drive
DRIVE_PATH = "G:/Meine Ablage/"


Export Funktion

In [15]:
def append_to_csv(df_new, filename):
    file_path = os.path.join(DRIVE_PATH, filename)

    try:
        # Falls die Datei bereits existiert, lese sie ein und hänge die neuen Daten an
        if os.path.exists(file_path):
            df_existing = pd.read_csv(file_path, sep=";", encoding="utf-8-sig", on_bad_lines="skip")
            df_combined = pd.concat([df_existing, df_new], ignore_index=True)
        else:
            df_combined = df_new  # Falls die Datei nicht existiert, erstelle sie neu

        # CSV speichern mit besserer Formatierung für Excel
        df_combined.to_csv(
            file_path,
            index=False,
            sep=";",  # Semikolon als Trennzeichen für bessere Excel-Kompatibilität
            encoding="utf-8-sig",  # Richtige Zeichenkodierung für Excel
            lineterminator="\n"  # Zeilenumbrüche korrekt speichern
        )
        print(f"Datei erfolgreich aktualisiert: {file_path}")

    except Exception as e:
        print(f"Fehler beim Speichern der Datei {filename}: {e}")

def export_to_drive(df_posts, df_comments):
    try:
        # Posts an bestehende Datei anhängen
        append_to_csv(df_posts, "reddit_posts.csv")
        # Kommentare an bestehende Datei anhängen
        append_to_csv(df_comments, "reddit_comments.csv")

    except Exception as e:
        print(f"Fehler beim Export: {e}")


Export Funnktion anwenden

In [16]:
# Export der bereinigten Daten
export_to_drive(df_posts_clean, df_comments_clean)

Datei erfolgreich aktualisiert: G:/Meine Ablage/reddit_posts.csv
Datei erfolgreich aktualisiert: G:/Meine Ablage/reddit_comments.csv
